In [ ]:
!pip install jschon

In [1]:
from jschon import create_catalog,JSON, JSONSchema
import json,os
import pathlib
import inspect



In [2]:
create_catalog('2020-12')


Catalog('catalog')

In [4]:
os.path.dirname(os.path.abspath("__file__"))


'd:\\Yanis\\FAC\\M1\\S2\\PSTL\\experiments\\validation'

In [35]:
src_file_path = os.path.dirname(os.path.abspath("__file__"))

# print(src_file_path)
dataset_dir =  src_file_path+'/dataset/1'
schema_dir = f'{dataset_dir}/schema.json'
instance_dir=f'{dataset_dir}/instance.json'

In [44]:
def get_errors_validation(schema_dir:str,instance_dir:str): 
    
    with open(schema_dir,'r',encoding='utf-8') as f : 
        schema= json.load(f)
    schema['$schema'] ="https://json-schema.org/draft/2020-12/schema"
    schema['$id'] ="https://jschon.dev/demo"
    demo_schema = JSONSchema(schema)
    # print(demo_schema)
    with open (instance_dir,'r',encoding='utf-8') as f : 
        instance=json.load(f)
        
    result = demo_schema.evaluate(
        JSON(instance)
    )
    print(result.output('detailed')['errors'])
    return result.output('detailed')

In [45]:

get_errors_validation(schema_dir,instance_dir)

[{'instanceLocation': '/name', 'keywordLocation': '/properties/name/anyOf', 'absoluteKeywordLocation': 'https://jschon.dev/demo#/properties/name/anyOf', 'errors': [{'instanceLocation': '/name', 'keywordLocation': '/properties/name/anyOf/0/required', 'absoluteKeywordLocation': 'https://jschon.dev/demo#/properties/name/anyOf/0/required', 'error': "The object is missing required properties ['a']"}, {'instanceLocation': '/name', 'keywordLocation': '/properties/name/anyOf/1/type', 'absoluteKeywordLocation': 'https://jschon.dev/demo#/properties/name/anyOf/1/type', 'error': 'The instance must be of type "integer"'}]}]


{'valid': False,
 'instanceLocation': '',
 'keywordLocation': '',
 'absoluteKeywordLocation': 'https://jschon.dev/demo#',
 'errors': [{'instanceLocation': '/name',
   'keywordLocation': '/properties/name/anyOf',
   'absoluteKeywordLocation': 'https://jschon.dev/demo#/properties/name/anyOf',
   'errors': [{'instanceLocation': '/name',
     'keywordLocation': '/properties/name/anyOf/0/required',
     'absoluteKeywordLocation': 'https://jschon.dev/demo#/properties/name/anyOf/0/required',
     'error': "The object is missing required properties ['a']"},
    {'instanceLocation': '/name',
     'keywordLocation': '/properties/name/anyOf/1/type',
     'absoluteKeywordLocation': 'https://jschon.dev/demo#/properties/name/anyOf/1/type',
     'error': 'The instance must be of type "integer"'}]}]}